In [ ]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score,precision_score, recall_score, roc_auc_score
from sklearn.decomposition import PCA
from sklearn.utils import shuffle
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.mixture import GaussianMixture
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from sklearn.impute import SimpleImputer
import tensorflow.keras
import keras.metrics
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.callbacks import History 
from keras.utils import plot_model
from keras.optimizers import SGD
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFECV, RFE


In [19]:
path = "PTSD.xlsx"
df = pd.read_excel(path)
df = df[~df["PCL_Strict3"].isna()]


In [20]:
features = ["age", "highschool_diploma",  "dyslexia", "ADHD", "T1Acc1t", "T1Acc1n", "T1bias", "phq1", "lot1", "trait1",
               "state1", "PCL1",  "PCL_Broad1", "PCL_Strict1", "phq2", "lot2", "trait2", "state2", "PCL2", "PCL_Broad2", 
                 "PCL_Strict2", "cd_risc1", "active_coping1", "planning1", "positive_reframing1", "acceptance1", "humor1",
                 "religion1", "emotional_support1","instrumental_support1", "self_distraction1", "denial1", 
               "venting1", "substance_use1", "behavioral_disengagement1", "self_blame1", "active_coping2", "planning2",
                "positive_reframing2", "acceptance2", "humor2", "religion2", "emotional_support2", "instrumental_support2", 
                 "self_distraction2", "denial2", "venting2", "substance_use2", "behavioral_disengagement2", "self_blame2",
                 "trauma_history8_1", "HML_5HTT", "HL_MAOA", "HML_NPY", "COMT_Ranked", "COMT_Hap1_recode", 
               "COMT_Hap2_recode", "COMT_Hap1_LvsMH", "HML_FKBP5", "bad_features"]

In [21]:
bad_features = ["T1ETBE", "T1bias", "state1", "state2", "trait1", "trait2", "phq1", "phq2", "PCL1", "PCL2",
                "denial1", "substance_use1", "self_blame1", "denial2", "substance_use2","self_blame2",
                         "trauma_history8_1"]
numerical_features = [ "T1ETBE", "T1Acc1t", "T1Acc1n", "T1bias", "T2Acc1t","T2Acc1n", "T2bias", "state1", "state2",
                        "trait1", "trait2", "lot1", "lot2", "phq1", "phq2", "cd_risc1", "PCL1", "PCL2", "bad_features"]
categorical_features = ["age", "highschool_diploma",  "dyslexia", "ADHD", "T1Acc1t", "T1Acc1n", "T1bias", "phq1", "lot1", "trait1",
               "state1", "PCL1",  "PCL_Broad1", "PCL_Strict1", "phq2", "lot2", "trait2", "state2", "PCL2", "PCL_Broad2", 
                 "PCL_Strict2", "cd_risc1", "active_coping1", "planning1", "positive_reframing1", "acceptance1", "humor1",
                 "religion1", "emotional_support1","instrumental_support1", "self_distraction1", "denial1", 
               "venting1", "substance_use1", "behavioral_disengagement1", "self_blame1", "active_coping2", "planning2",
                "positive_reframing2", "acceptance2", "humor2", "religion2", "emotional_support2", "instrumental_support2", 
                 "self_distraction2", "denial2", "venting2", "substance_use2", "behavioral_disengagement2", "self_blame2",
                 "trauma_history8_1", "HML_5HTT", "HL_MAOA", "HML_NPY", "COMT_Ranked", "COMT_Hap1_recode", 
               "COMT_Hap2_recode", "COMT_Hap1_LvsMH", "HML_FKBP5", "Ashken_scale", "Sephar_scale", "Unknown"]

df['bad_features'] = (df > df.mean())[bad_features].sum(axis=1)

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
df[numerical_features] = imp.fit_transform(df[numerical_features])

imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
df[categorical_features] = imp.fit_transform(df[categorical_features])

pca = PCA(n_components = 1)
df["pcls"] = pca.fit_transform(df[["PCL1",  "PCL_Broad1", "PCL_Strict1"]])
df.drop(["PCL1",  "PCL_Broad1", "PCL_Strict1"], axis=1, inplace=True)
features.append("pcls")
features.remove("PCL1")
features.remove("PCL_Broad1")
features.remove("PCL_Strict1")


X = df[features]


ss = StandardScaler()
X = ss.fit_transform(X)


Y = df["PCL_Strict3"]

In [22]:
rand_item = eval("['PCL2', 'positive_reframing1', 'trauma_history8_1', 'lot2', 'T1bias', 'emotional_support1', 'self_blame1', 'PCL2']")

pca = PCA(n_components = 2)
data_transformed = df[rand_item]
data_transformed = pca.fit_transform(data_transformed)

db = DBSCAN(eps=0.5, min_samples=10, leaf_size=30).fit(X)
X = np.hstack((db.labels_.reshape(-1,1), X))


rand_item = eval("['instrumental_support1', 'COMT_Hap1_recode', 'dyslexia', 'HL_MAOA', 'lot1', 'active_coping2', 'ADHD']")

pca = PCA(n_components = 2)
data_transformed = df[rand_item]
data_transformed = pca.fit_transform(data_transformed)

db = DBSCAN(eps=0.7, min_samples=10, leaf_size=30).fit(X)
X = np.hstack((db.labels_.reshape(-1,1), X))


rand_item = eval("['COMT_Hap1_LvsMH', 'venting2', 'positive_reframing2', 'COMT_Ranked', 'behavioral_disengagement2', 'trait1', 'HL_MAOA', 'HL_MAOA']")

pca = PCA(n_components = 2)
data_transformed = df[rand_item]
data_transformed = pca.fit_transform(data_transformed)

db = DBSCAN(eps=0.5, min_samples=10, leaf_size=30).fit(X)
X = np.hstack((db.labels_.reshape(-1,1), X))


rand_item = eval("['PCL_Strict2', 'emotional_support2', 'PCL_Broad2', 'emotional_support1', 'COMT_Hap1_LvsMH', 'HML_FKBP5', 'lot1', 'COMT_Hap1_LvsMH', 'COMT_Hap2_recode', 'state1']")

pca = PCA(n_components = 2)
data_transformed = df[rand_item]
data_transformed = pca.fit_transform(data_transformed)

db = DBSCAN(eps=0.6, min_samples=10, leaf_size=30).fit(X)
X = np.hstack((db.labels_.reshape(-1,1), X))


rand_item = eval("['phq2', 'substance_use1', 'instrumental_support2', 'substance_use1', 'trait1']")

pca = PCA(n_components = 2)
data_transformed = df[rand_item]
data_transformed = pca.fit_transform(data_transformed)

db = DBSCAN(eps=0.5, min_samples=5, leaf_size=30).fit(X)
X = np.hstack((db.labels_.reshape(-1,1), X))


In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.1, random_state=271828, stratify=Y)
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_train, y_train, test_size = 0.1, random_state=271828, stratify=y_train)


In [26]:
def create_data(X_train, y_train):
    X_train_3 = X_train[y_train==1]
    y_train_3 = y_train[y_train==1]
    X_train_4 = X_train[y_train==0][:15:]
    y_train_4 = y_train[y_train==0][:15:]
    X_train_5 = np.vstack((X_train_4, X_train_3))
    y_train_5 =  np.hstack((y_train_4, y_train_3))
    sm = SMOTE(random_state=27)
    X_train_6, y_train_6 = sm.fit_sample(X_train_5, y_train_5.ravel())
    X_train_6 = X_train_6[y_train_6==0]
    y_train_6 = y_train_6[y_train_6==0]
    return X_train_6, y_train_6

In [27]:
kfold = StratifiedKFold(n_splits=5, shuffle=True)
cvscores = []
y_train_2 = np.array(y_train_2)
X_train_2 = np.array(X_train_2)
for rfe_n in [10, 16]:
    for first_layer in [60, 50, 70]:
        for loops in [76]:
            for each_layer in [7]:
                num_layers = 3
                first_layer = first_layer
                each_layer = each_layer
                num_smote = 1
                loops = loops
                scores = []
                print("num_layers", num_layers, "\nfirst_layer", first_layer, 
                          "\neach_layer", each_layer, "\nnum_smote", num_smote, "\nloops", loops, "\nrfe_n", rfe_n)
                for train, test in kfold.split(X_train_2, y_train_2):

                    rfe =  RFE(RandomForestClassifier(n_estimators=100), rfe_n).fit(X_train_2[train], y_train_2[train])
                    X = rfe.transform(X_train_2[train])
                    x, y = create_data(X, y_train_2[train])
                    created_X = x
                    created_y = y
                    for i in range(loops):
                        X_train_3, y_train_3 = shuffle(X, y_train_2[train])
                        x, y = create_data(X_train_3, y_train_3)
                        created_X = np.vstack((created_X, x))
                        created_y = np.hstack((created_y, y))
                    X_train_3 = np.vstack([created_X, X_train_3])
                    y_train_3 =  np.hstack([created_y, y_train_3])

                    sm = SMOTE(random_state=27)
                    X_train_res, y_train_res = sm.fit_sample(X_train_3, y_train_3.ravel())

                  # create model
                    n_cols = X_train_res.shape[1]
                    model = Sequential()
                    model.add(Dense(first_layer, activation='elu', input_dim = n_cols))
                    model.add(Dropout(0.5))

                    for i in range(num_layers):
                        model.add(Dense(each_layer, activation='elu'))
                        model.add(Dropout(0.5))

                    model.add(Dense(1, activation='sigmoid'))

                    model.compile(optimizer='adam', 
                                  loss='binary_crossentropy')


                    # Fit the model
                    model.fit(X_train_res, y_train_res, epochs = 250, validation_split = .1, verbose=0)
                    # evaluate the model
                    X_test_rfe = rfe.transform(X_train_2[test])
                    y_pred =  model.predict(X_test_rfe)
                    y_pred = y_pred>0.5

                    s = precision_score(y_pred,y_train_2[test])
                    print("precision", s)
                    print("recall", recall_score(y_pred,y_train_2[test]))
                    scores.append(s)
                print("mean precision scores", np.mean(scores))

num_layers 3 
first_layer 60 
each_layer 7 
num_smote 1 
loops 76 
rfe_n 10
precision 0.375
recall 0.14285714285714285
precision 0.5714285714285714
recall 0.15384615384615385
precision 0.0
recall 0.0
precision 0.42857142857142855
recall 0.13043478260869565
precision 0.14285714285714285
recall 0.09090909090909091
mean precision scores 0.30357142857142855
num_layers 3 
first_layer 50 
each_layer 7 
num_smote 1 
loops 76 
rfe_n 10
precision 0.375
recall 0.13043478260869565
precision 0.2857142857142857
recall 0.10526315789473684
precision 0.14285714285714285
recall 0.058823529411764705
precision 0.42857142857142855
recall 0.15789473684210525
precision 0.2857142857142857
recall 0.10526315789473684
mean precision scores 0.30357142857142855
num_layers 3 
first_layer 70 
each_layer 7 
num_smote 1 
loops 76 
rfe_n 10
precision 0.25
recall 0.13333333333333333
precision 0.14285714285714285
recall 0.07692307692307693
precision 0.2857142857142857
recall 0.09523809523809523
precision 0.4285714285714

In [ ]:
y_train_2[1]